# K-nearest-neighbours with sci-kit learn

W tym notebooku będziemy używać interfejsu Sci-kit learn by stworzyć klasyfikator KNN, na zbiorze danych Iris, oraz dopasować jego parametry.

Notebook będzie obejmował następujące kroki:

1. Załadowanie zbioru danych z pliku .csv
2. Podział na zbiór treningowy i testowy
3. Utworzenie podstawowego modelu K-nn oraz sprawdzenie jego trafności
4. Poszukiwanie najlepszego zestawu parametrów za pomocą metody Grid-Search
5. Analiza błędu na zbiorze testowym

# Przygotowanie

## import bibliotek

In [4]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


## Ładowanie danych z pliku CSV

Użyjemy biblioteki Pandas i jej wbudowanej funkcjonalności do ładowania dnanych. 

```{python}

pandas.read_csv(....)

```

Załadowanie zbioru danych:

In [5]:
iris = pd.read_csv("./iris.csv")

Sprawdźmy pierwsze pięć elementów

In [6]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


**Species** czyli **gatunek irysa** jest naszą zmienną odpowiedzi - zmienną zależną

## Podział zbioru na dane treningowe i testowe

Teraz, gdy mamy zbiór danych wczytany do pamięci, możemy go podzielić na podzbiór:

**treningowy** - który używany będzie do szkolenia algorytmu

**testowy** - który zostawimy na sam koniec, żeby sprawdzić, jak dobrze sprawuje się algorytm


Żeby to zrobić musimy najpierw rozdzielić zmienną zależną *y* (zmienną odpowiedzi) od pozostałych zmiennych *X* - niezależnych.
Potem uruchomimy funkcję *train_test_split* z pakietu sklearn.

[http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

Wynikiem jej działania są 4 zmienne:
* X training
* X testing
* y training
* y testing



Stwórzymy zmienną X, która będzie zawierać 4 atrybuty

Stwórzymy zmienną y, która będzie wektorem odpowiedzi, zawierającym klasę docelową

In [9]:
X = iris.drop(columns='species', axis=1)
y = iris.species

In [10]:
print(X.head())
print(y.head())

   sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2
0    setosa
1    setosa
2    setosa
3    setosa
4    setosa
Name: species, dtype: object


20% danych będzie przeznaczone do testowania, reszta - na trening

In [20]:
TEST_FRACTION = 0.2
RANDOM_STATE = 123

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_FRACTION, random_state=RANDOM_STATE)

In [21]:
print(X_train.shape)
print(X_test.shape)

(120, 4)
(30, 4)


# Szkolenie modelu

## Stwórzymy podstawowy model KNN

Teraz możemy utworzyć podstawowy model KNN - *K-nearest neighbors classifier* używając implementacji z Sklearn'a.

Na początku będziemy używać DOMYŚLNYCH PARAMETRÓW


* Najpierw - importujemy model z biblioteki
* Potem tworzymy **instancję** klasy - czyli konkretny egzemplarz abstrakcyjnego algorytmu - obiekt
* Po trzecie - używamy funkcji **fit**, która dopasuje go do danych. Do funkcji przekazujemy **X_train, y_train**, jako materiał do nauki

In [23]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier()
neigh.fit( X_train, y_train )

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [24]:
print(neigh)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')


Teraz możemy dokonać predykcji za pomocą algorytmu na danych testowych. W tym celu wykonujemy funkcję

```{python}

knn.predict( .... )

```

In [25]:
predictions = neigh.predict( X_test )

In [26]:
expected_actual = pd.DataFrame({'prawdziwe': y_test, 'przewidziane': predictions})
expected_actual

,prawdziwe,przewidziane
72,versicolor,virginica
112,virginica,virginica
132,virginica,virginica
88,versicolor,versicolor
37,setosa,setosa
138,virginica,virginica
87,versicolor,versicolor
42,setosa,setosa
8,setosa,setosa
90,versicolor,versicolor


Teraz możemy policzyć trafność. Jest to odsetek poprawnych klasfikacji w całym zbiorze.

Każda metryka w Pythonie działa tak samo. Przekazujemy jej wartości oczekiwane i wartości przewidziane:

```{python}

metric(y_true=...., y_test=...)


```

In [27]:
metrics.accuracy_score(  y_true=y_test, y_pred=predictions  )

0.9666666666666667

W zbiorze danych mamy 3 różne gatunki irysa. Trafność to miara **średnia**. Zobaczmy jak wygląda ogólna metryka dla całej klasyfikacji. Służy do tego funkcja **classification_report**

In [28]:
print(metrics.classification_report( y_true=y_test, y_pred=predictions ))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       1.00      0.83      0.91         6
   virginica       0.92      1.00      0.96        11

   micro avg       0.97      0.97      0.97        30
   macro avg       0.97      0.94      0.96        30
weighted avg       0.97      0.97      0.97        30



## Poszukiwanie najlepszych parametrów dla modelu z wykorzystaniem funkcji Grid Search

Osiągnęliśmy bardzo dobry wynik z klasyfikatorem na domyślnych parametrach. Możemy spróbować z innymi ustawieniami. Zazwyczaj jednak parametrów jest dużo i istnieje bardzo wiele kombinacji. Z tego względu istnieje półautomatyczna metoda poszukiwania najlepszych paramterów - **Grid Search** która przeszukuje "najbardziej obiecujące" zestawy. 

Grid search działa w sposób następujący:

* Przyjmuje jako parametr obiekt algorytmu oraz siatkę parametrów do przeszukania
* Podejmemy mu dodatkowe ustawienia jak np. liczba iteracji, etc.
* Podaejmy strategię przeszukiwania parametrów - kompletnie losowa/z użyciem optymalizacji/ genetyczna/ etc.



In [31]:
from sklearn.model_selection import GridSearchCV

Teraz ustawiamy parametry do przeszukania.

In [32]:
parameters = {
    
    # Liczba sąsiadów w algorytmie KNN
    'n_neighbors': [1, 2, 3, 4], 
    
    # Czy każdy sąsiad ma ważyć tyle samo, czy np. 
    # być oceniany pod kątem odległości?
    'weights': ['uniform', 'distance'],
    
    # Typ metryki odległości. Albo dystans
    # euklidesowy, albo uogólniona metryka Minskoswkiego
    'metric': ['minkowski', 'euclidean']

}

Teraz możemy utworzyć obiekt przeszukiwania paramterów, który **szkolimy jak klasyfikator**

In [34]:
knn = KNeighborsClassifier()
clf = GridSearchCV( knn, param_grid=parameters, cv=10 )
clf.fit( X, y )

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [1, 2, 3, 4], 'weights': ['uniform', 'distance'], 'metric': ['minkowski', 'euclidean']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Możemy teraz sprawdzić wyniki każej walidacji krzyżowej z osobna, tak samo jak najlepsze zestawy parametrów:

In [38]:
clf.cv_results_

C:\Users\fwojcik\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\fwojcik\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\fwojcik\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_

{'mean_fit_time': array([0.00139892, 0.00059667, 0.00157127, 0.0008975 , 0.        ,
        0.0015595 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00312402, 0.        , 0.        , 0.        ,
        0.00156202]),
 'mean_score_time': array([0.00128415, 0.00049839, 0.00196044, 0.00079844, 0.00312698,
        0.        , 0.00156491, 0.        , 0.        , 0.        ,
        0.00312393, 0.        , 0.00155938, 0.        , 0.00156231,
        0.00156205]),
 'mean_test_score': array([0.96      , 0.96      , 0.95333333, 0.96      , 0.96666667,
        0.96666667, 0.96666667, 0.96666667, 0.96      , 0.96      ,
        0.95333333, 0.96      , 0.96666667, 0.96666667, 0.96666667,
        0.96666667]),
 'mean_train_score': array([1.        , 1.        , 0.97851852, 1.        , 0.96074074,
        1.        , 0.9637037 , 1.        , 1.        , 1.        ,
        0.97851852, 1.        , 0.96074074, 1.        , 0.9637037 ,
        1.        ]),
 'param_metric': mask

In [35]:
pd.DataFrame(clf.cv_results_)

C:\Users\fwojcik\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\fwojcik\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\fwojcik\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_metric,param_n_neighbors,param_weights,params,rank_test_score,split0_test_score,...,split7_test_score,split7_train_score,split8_test_score,split8_train_score,split9_test_score,split9_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.001399,0.001284,0.960000,1.000000,minkowski,1,uniform,"{'metric': 'minkowski', 'n_neighbors': 1, 'wei...",9,1.0,...,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.000487,0.000453,0.053333,0.000000
1,0.000597,0.000498,0.960000,1.000000,minkowski,1,distance,"{'metric': 'minkowski', 'n_neighbors': 1, 'wei...",9,1.0,...,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.000795,0.000674,0.053333,0.000000
2,0.001571,0.001960,0.953333,0.978519,minkowski,2,uniform,"{'metric': 'minkowski', 'n_neighbors': 2, 'wei...",15,1.0,...,0.933333,0.977778,1.0,0.977778,1.0,0.977778,0.003743,0.004576,0.052068,0.005185
3,0.000898,0.000798,0.960000,1.000000,minkowski,2,distance,"{'metric': 'minkowski', 'n_neighbors': 2, 'wei...",9,1.0,...,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.000299,0.000399,0.053333,0.000000
4,0.000000,0.003127,0.966667,0.960741,minkowski,3,uniform,"{'metric': 'minkowski', 'n_neighbors': 3, 'wei...",1,1.0,...,1.000000,0.955556,1.0,0.955556,1.0,0.955556,0.000000,0.006254,0.044721,0.007444
5,0.001559,0.000000,0.966667,1.000000,minkowski,3,distance,"{'metric': 'minkowski', 'n_neighbors': 3, 'wei...",1,1.0,...,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.004678,0.000000,0.044721,0.000000
6,0.000000,0.001565,0.966667,0.963704,minkowski,4,uniform,"{'metric': 'minkowski', 'n_neighbors': 4, 'wei...",1,1.0,...,1.000000,0.962963,1.0,0.955556,1.0,0.970370,0.000000,0.004695,0.044721,0.006988
7,0.000000,0.000000,0.966667,1.000000,minkowski,4,distance,"{'metric': 'minkowski', 'n_neighbors': 4, 'wei...",1,1.0,...,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.000000,0.000000,0.044721,0.000000
8,0.000000,0.000000,0.960000,1.000000,euclidean,1,uniform,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",9,1.0,...,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.000000,0.000000,0.053333,0.000000
9,0.000000,0.000000,0.960000,1.000000,euclidean,1,distance,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",9,1.0,...,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.000000,0.000000,0.053333,0.000000


Możemy też użyć najlepszego klasyfikatora, który został wyszkolony podczas poszukiwania parametrów.

In [39]:
clf.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [42]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
pd.DataFrame(ss.fit_transform(X))

,0,1,2,3
0,-0.900681,1.032057,-1.341272,-1.312977
1,-1.143017,-0.124958,-1.341272,-1.312977
2,-1.385353,0.337848,-1.398138,-1.312977
3,-1.506521,0.106445,-1.284407,-1.312977
4,-1.021849,1.263460,-1.341272,-1.312977
5,-0.537178,1.957669,-1.170675,-1.050031
6,-1.506521,0.800654,-1.341272,-1.181504
7,-1.021849,0.800654,-1.284407,-1.312977
8,-1.748856,-0.356361,-1.341272,-1.312977
9,-1.143017,0.106445,-1.284407,-1.444450


In [37]:
# Predict using best grid search results
predictions_grid_search = clf.predict( X_test )

# Print accuracy
metrics.accuracy_score( y_pred=predictions_grid_search, y_true=y_test )

0.9333333333333333